**Клонирование репозитония**

In [53]:
!rm -R rubert-turbo-payments

rm: cannot remove 'rubert-turbo-payments': No such file or directory


In [54]:
!git clone https://github.com/pyramidheadshark/rubert-turbo-payments.git

Cloning into 'rubert-turbo-payments'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 262 (delta 48), reused 76 (delta 29), pack-reused 152 (from 1)
Receiving objects: 100% (262/262), 47.04 MiB | 28.69 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [55]:
!ls rubert-turbo-payments/data/processed

dataset.csv		 dataset_oversampled.tsv  previous_categorized
dataset.tsv		 payments_main.tsv	  val_data.tsv
dataset_oversampled.csv  payments_training.tsv


### Тренировка модели

In [56]:
!pip install datasets > None
!pip install onnxruntime > None
!pip install optimum > None
!pip install onnx > None

In [57]:
!head rubert-turbo-payments/data/processed/dataset.tsv

За тур.поездку по договору №001 от 27.01.2023г	SERVICE
За оказание услуг по договору №53Б-02746 от 23/08/2024	SERVICE
Оплата штрафа	TAX
Лечение по договору №Д-00359/24 от 08.03.2025	SERVICE
"Оплата основного долга за период с 16.12.2024г. по 20 марта 2026г по Договору №Г03388/25 от 02.09.2027г с ОАО ""Второй банк"""	BANK_SERVICE
Оплата за Бульон Роллтон Домашний куриный 90г по счету 27336253164249532548 от 28.05.2023г Сумма 4210.00	FOOD_GOODS
Комиссионное вознаграждение за валютный перевод.	BANK_SERVICE
государственная пошлина	TAX
Лечение по договору №Д00184/63 от 27.12.2023 г.	SERVICE
Оплата по счету 0187,0188,0189 от 02.01.2024 (в рамках договора финансовой аренды №00438Б от 22 декабря 2025г). В т.ч. НДС(20%) 218333,33	LEASING


In [58]:
import wandb

wandb.login(key="-")

wandb: Currently logged in as: bratakmckey (bratakmckey-parsub). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [59]:
!python3 rubert-turbo-payments/src/models_dev/train.py --file_path rubert-turbo-payments/data/processed/dataset.tsv \
                                    --model_name sergeyzh/rubert-tiny-turbo \
                                    --output_dir ./results \
                                    --batch_size 512 \
                                    --epochs 70 \
                                    --learning_rate 5e-5 \
                                    --save_path rubert-turbo-payments/models/trained_model

/kaggle/working/rubert-turbo-payments/rubert-turbo-payments/src/models_dev/train.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].astype('category').cat.codes
/kaggle/working/rubert-turbo-payments/rubert-turbo-payments/src/models_dev/train.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['category'].astype('category').cat.codes
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sergeyzh/rubert-tiny-turbo

### Конвертация в onnx

In [60]:
!python3 rubert-turbo-payments/src/models_dev/convert_to_onnx.py \
                            --model_path rubert-turbo-payments/models/trained_model \
                            --onnx_path rubert-turbo-payments/models/onnx_model

Конвертация модели в формат ONNX...
The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
Модель успешно сохранена в формате ONNX: rubert-turbo-payments/models/onnx_model


### Квантирование

In [61]:
!python3 rubert-turbo-payments/src/models_dev/quantize.py \
                                --onnx_model_path rubert-turbo-payments/models/onnx_model/model.onnx \
                                --save_dir rubert-turbo-payments/models/onnx_model \
                                --max_length 128

Модель квантована и сохранена: rubert-turbo-payments/models/onnx_model/model_quantized.onnx


### Тестирование скорости инференса

In [33]:
!python3 rubert-turbo-payments/src/models_dev/q_infer.py \
                    --file_path rubert-turbo-payments/data/raw/payments_main.tsv \
                 --model_path rubert-turbo-payments/models/onnx_model/model_quantized.onnx \
                 --tokenizer_path rubert-turbo-payments/models/onnx_model \
                 --output_file rubert-turbo-payments/data/predictions.tsv \
                 --batch_size 64 \
                 --max_length 128

Inferencing: 100%|████████████████████████████| 391/391 [01:01<00:00,  6.38it/s]
Predictions saved to rubert-turbo-payments/data/predictions.tsv


In [63]:
!zip -r models.zip rubert-turbo-payments/models/onnx_model

  adding: rubert-turbo-payments/models/onnx_model/ (stored 0%)
  adding: rubert-turbo-payments/models/onnx_model/tokenizer_config.json (deflated 73%)
  adding: rubert-turbo-payments/models/onnx_model/special_tokens_map.json (deflated 80%)
  adding: rubert-turbo-payments/models/onnx_model/model.onnx (deflated 8%)
  adding: rubert-turbo-payments/models/onnx_model/tokenizer.json (deflated 73%)
  adding: rubert-turbo-payments/models/onnx_model/config.json (deflated 55%)
  adding: rubert-turbo-payments/models/onnx_model/model_quantized.onnx (deflated 39%)
  adding: rubert-turbo-payments/models/onnx_model/vocab.txt (deflated 64%)


### Тестирование точности квантированной модели

In [64]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("rubert-turbo-payments/data/processed/dataset.tsv", sep='\t', header=None, names=['description', 'category'])
train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['category'], random_state=42)

# Сохранение val_data для тестирования
val_data.to_csv("rubert-turbo-payments/data/processed/val_data_dataset.tsv", sep='\t', index=False)

In [65]:
!python3 rubert-turbo-payments/src/models_dev/q_infer_val.py \
    --val_data_path rubert-turbo-payments/data/processed/val_data_dataset.tsv \
    --model_path rubert-turbo-payments/models/onnx_model/model_quantized.onnx \
    --tokenizer_path rubert-turbo-payments/models/onnx_model \
    --batch_size 64 \
    --max_length 128

Inferencing: 100%|██████████████████████████████| 30/30 [00:05<00:00,  5.20it/s]
True labels: [7, 1, 2, 1, 2, 5, 6, 3, 3, 4]
Predictions: [7, 1, 2, 1, 2, 5, 6, 3, 3, 7]
Validation Accuracy: 0.9168
Validation F1 Score: 0.9159
